# Task 1: masked language model

## 1. pkgs loading

In [1]:
import os
# os.environ["WORLD_SIZE"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset
import torch
import evaluate

## 2. load data

raw dataset can be downloaded from `https://github.com/zyds/transformers-code/tree/master/02-NLP%20Tasks/14-language_model/wiki_cn_filtered`

In [2]:
ds = Dataset.load_from_disk('data/wiki_cn_filtered/')
datasets = ds.train_test_split(test_size=0.1)

In [3]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## 3. data preprocessing

In [4]:
ckpt = 'hfl/chinese-macbert-base'
tokenizer = AutoTokenizer.from_pretrained(ckpt)

def process_function(examples):
    return tokenizer(examples['completion'], truncation=True, max_length=384)

tokenized_datasets = datasets.map(process_function, batched=True, num_proc=4, remove_columns=ds.column_names)
tokenized_datasets

Map (num_proc=4):   0%|          | 0/9000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [5]:
from torch.utils.data import DataLoader
dl = DataLoader(tokenized_datasets['train'], batch_size=8, shuffle=False, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15))

In [6]:
batch = next(enumerate(dl))[1]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [7]:
batch['labels'][0][15:22]

tensor([-100, -100, 8024, -100, 1399, -100, -100])

In [8]:
batch['input_ids'][0][15:22]

tensor([2157,  518,  103, 1333,  103, 8038,  517])

In [9]:
tokenizer.decode(batch['input_ids'][0][15:22])

'家 》 [MASK] 原 [MASK] ： 《'

## 4. create model

In [10]:
model = AutoModelForMaskedLM.from_pretrained(ckpt)

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 5. config training args

In [ ]:
args = TrainingArguments(output_dir='./masked_lm', 
                            per_device_train_batch_size=32,
                            per_device_eval_batch_size=64,
                            num_train_epochs=1,
                            logging_steps=10,
                            load_best_model_at_end=True,
                            evaluation_strategy='epoch',
                            save_strategy='epoch',
                            save_total_limit=2,
                            report_to='none',
                            )
trainer = Trainer(model=model, args=args, train_dataset= tokenized_datasets['train'], eval_dataset=tokenized_datasets['test'],
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15))

## 6. train

In [ ]:
trainer.train()

/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,1.298100,1.217772


TrainOutput(global_step=141, training_loss=1.3259769179296832, metrics={'train_runtime': 129.9371, 'train_samples_per_second': 69.264, 'train_steps_per_second': 1.085, 'total_flos': 1776437692416000.0, 'train_loss': 1.3259769179296832, 'epoch': 1.0})

## 7. inference

In [ ]:
from transformers import pipeline
pipe = pipeline('fill-mask', model=model, tokenizer=tokenizer, device=0)
pipe("西安交通大[MASK]博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆")

[{'score': 0.9999794960021973,
  'token': 2110,
  'token_str': '学',
  'sequence': "西 安 交 通 大 学 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆"},
 {'score': 1.3157555258658249e-05,
  'token': 2119,
  'token_str': '學',
  'sequence': "西 安 交 通 大 學 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆"},
 {'score': 2.415609515082906e-06,
  'token': 1920,
  'token_str': '大',
  'sequence': "西 安 交 通 大 大 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆"},
 {'score': 1.4857732821837999e-06,
  'token': 7368,
  'token_str': '院',
  'sequence': "西 安 交 通 大 院 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆"},
 {'score': 4.68710368295433e-07,
  'token': 6887,
  'token_str': '道',
  'sequence': "西 安 交 通 大 道 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆"}]

In [ ]:
results = pipe("西安交通大[MASK][MASK]物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆")

In [ ]:
results[0]

[{'score': 0.9998903274536133,
  'token': 2110,
  'token_str': '学',
  'sequence': "[CLS] 西 安 交 通 大 学 [MASK] 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
 {'score': 4.174594869255088e-05,
  'token': 2119,
  'token_str': '學',
  'sequence': "[CLS] 西 安 交 通 大 學 [MASK] 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
 {'score': 4.001786874141544e-05,
  'token': 1920,
  'token_str': '大',
  'sequence': "[CLS] 西 安 交 通 大 大 [MASK] 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
 {'score': 7.771494892949704e-06,
  'token': 7368,
  'token_str': '院',
  'sequence': "[CLS] 西 安 交 通 大 院 [MASK] 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
 {'score': 1.990778855542885e-06,
  'token': 1336,
  'token_str': '厦',
  'sequence': "[CLS] 西 安 交 通 大 厦 [MASK] 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"}]

In [ ]:
results[1]

[{'score': 0.9943378567695618,
  'token': 1300,
  'token_str': '博',
  'sequence': "[CLS] 西 安 交 通 大 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
 {'score': 0.003546401159837842,
  'token': 1745,
  'token_str': '图',
  'sequence': "[CLS] 西 安 交 通 大 [MASK] 图 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
 {'score': 0.0011761495843529701,
  'token': 3152,
  'token_str': '文',
  'sequence': "[CLS] 西 安 交 通 大 [MASK] 文 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
 {'score': 0.00017536857922095805,
  'token': 5279,
  'token_str': '纪',
  'sequence': "[CLS] 西 安 交 通 大 [MASK] 纪 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
 {'score': 8.50768992677331e-05,
  'token': 1920,
  'token_str': '大',
  'sequence': "[CLS] 西 安 交 通 大 [MASK] 大 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"}]